# Step 1 - Mount drive
Mount your google drive which has the training, test and dev jsonlfiles and labels

In [48]:
# from google.colab import drive
# drive.flush_and_unmount()

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p /content/drive2
!google-drive-ocamlfuse /content/drive2

In [ ]:
%cd /content/drive2//NLP_data
%ls

# Step 2 - Setup

##Load Essential Libraries

In [ ]:
!pip install spacy
!pip install -U pip setuptools wheel
!python -m spacy download en_core_web_sm
!pip install pyLDAvis
!python -m pip install wordcloud
!pip install textblob
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [ ]:
import os
import re
import string
from tqdm import tqdm
import numpy as np
import pandas as pd
from datetime import datetime, time
import time
import collections
import random
import json
import math
from fractions import gcd
from functools import reduce 
import operator
import itertools
import networkx as nx
%matplotlib inline

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import trigrams
from nltk.corpus import stopwords
from nltk import bigrams
from nltk.util import ngrams

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns

# import wordcloud
from wordcloud import WordCloud
from textblob import TextBlob
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim import corpora
import pickle
import gensim

import spacy
spacy.load("en_core_web_sm")
from spacy.lang.en import English
import spacy
import en_core_web_sm

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

## Load Dataset



In [ ]:
def user_details(tweets_final,tweets_df):
    tweets_final["screen_name"] = tweets_df["user"].apply(lambda x: x["screen_name"])
    tweets_final["user_id"] = tweets_df["user"].apply(lambda x: x["id"])
    tweets_final["followers_count"] = tweets_df["user"].apply(lambda x: x["followers_count"])
    tweets_final["friends_count"] = tweets_df["user"].apply(lambda x: x["friends_count"])
    tweets_final["statuses_count"] = tweets_df["user"].apply(lambda x: x["statuses_count"])
    tweets_final["listed_count"] = tweets_df["user"].apply(lambda x: x["listed_count"])
    tweets_final["favourites_count"] = tweets_df["user"].apply(lambda x: x["favourites_count"])
    tweets_final["verified"] = tweets_df["user"].apply(lambda x: x["verified"])
    tweets_final["location"] = tweets_df["user"].apply(lambda x: x["location"])
    tweets_final["time_zone"] = tweets_df["user"].apply(lambda x: x["time_zone"])
    tweets_final["geo_enabled"] = tweets_df["user"].apply(lambda x: x["geo_enabled"])
    tweets_final["user_created_at"] = tweets_df["user"].apply(lambda x: x["created_at"])
    tweets_final["protected"] = tweets_df["user"].apply(lambda x: x["protected"])
    tweets_final["default_profile"] = tweets_df["user"].apply(lambda x: x["default_profile"])

    return tweets_final
def get_basics(tweets_final,tweets_df):
    # Just copy the 'in_reply' columns to the new dataframe
    tweets_final["id"]= tweets_df["id_str"]
    tweets_final["in_reply_to_status_id_str"]= tweets_df["in_reply_to_status_id_str"]
    tweets_final["tweet"]= tweets_df["tweet"]
    tweets_final["in_reply_to_user_id_str"]= tweets_df["in_reply_to_user_id_str"]
    tweets_final["in_reply_to_screen_name"] = tweets_df["in_reply_to_screen_name"]
    tweets_final["favorited"] = tweets_df["favorited"]
    tweets_final["favorite_count"]= tweets_df["favorite_count"]
    tweets_final["retweeted"]= tweets_df["retweeted"]
    tweets_final["retweet_count"]= tweets_df["retweet_count"]
    tweets_final["created_at"]= tweets_df["created_at"]

    return tweets_final

def fill_df(tweets_final,tweets_df):
    user_details(tweets_final,tweets_df)
    get_basics(tweets_final,tweets_df)
    return tweets_final

In [ ]:
#function to return the replies as a list for a source tweet 
def get_replies(df):
    dict_time = {}
    # rows = df.loc["Utah Jazz"]
    for index, row in df.iterrows():
        #check if the in_reply_to_status_id_str string matches a number
        x = re.search(r'\d+', str(row['in_reply_to_status_id_str']))  
        if x:
            
            #append the reply tweets in a big_list and return the list
            big_list.append(row["tweet"]) 
            #if the tweets are created at the same time - append it to existing list
            if (row["created_at"] in dict_time):
                # print("True")
                value = dict_time.get(row["created_at"])
                value.append(row["tweet"])
                dict_time[row["created_at"]] = value
                # print(replyTw_list)
            else:
                replyTw_list = []
                replyTw_list.append(row["tweet"])
                # print(replyTw_list) 
                dict_time[row["created_at"]] = replyTw_list
            # print(dict_time)
        else:
            big_list = []
    #returns the list of replies for a source tweet, length of the reply list and dict of {created_at_Time:reply_tweet}
    # print(big_list, dict_time)
    return big_list,len(big_list),dict_time 

    
#function to return the list of reply tweets sorted by time for all source tweets
#takes the list of dictionaries as a parameter of the form [{created_At_time1:[tweet1, tweet2]},{created_At_time2:[tweet3]}]
def get_sorted_by_time(dict_replies_list):
    sorted_list = []
    for i in range(len(dict_replies_list)):
        # dtime = 'Mon Dec 15 21:50:30 +0000 2014'
        newkeys_list = []
        vals =[]
        for k, v in dict_replies_list[i].items():  
            new_datetime = datetime.strftime(datetime.strptime(k,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S') 
            date_to_float = datetime.fromisoformat(new_datetime).timestamp()#1420646211.0
            newkeys_list.append(date_to_float) #list of keys for the new dictionary 
            vals = list(dict_replies_list[i].values()) #list of values for the new dictionary 
        newdictionary = {k: v for k, v in zip(newkeys_list, vals)} #zip the keys and values to form a new dictionary 
        od = collections.OrderedDict(sorted(newdictionary.items()))
#         print("DICT KEYS AFTER SORtiNG:\n", od.keys(), len(od))
#         print("DICT KEYS AFTER SORtiNG:\n", newdictionary.keys(), len(newdictionary),"\n Values:\n",newdictionary.values())
        sorted_list.append(od)
    # print("The length of sorted list is = ", len(sorted_list))
    return sorted_list

### 2.1. Load Train Data
The train.data.jsonl has about 4k source tweets and about 81k source + reply tweets.

The label files ([train,dev].label.json), are standard JSON files:
{
"552800070199148544": "non-rumour",
"544388259359387648": "non-rumour",
"552805970536333314": "non-rumour",
"525071376084791297": "rumour"
} 

All data files ([train,dev,test,covid].data.jsonl) are JSONL files, where each line is a JSON string. For these files, each line is an event: a list of tweets where the first tweet is a source tweet and the rest are reply tweets.

Here the train data is loaded and merged with the labels and the resulting columns are "id_str","in_reply_to_status_id_str","text", "user","favorited","in_reply_to_user_id_str","created_at", "label"].

In [ ]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_rows', 500)
with open("train.data.jsonl", 'r', encoding='utf-8') as train_data_file:
  list_of_rlists = []
  chunks = [] #list of all the dataframes for all source and reply tweets
  num_replies = [] #list of number of replies for every source tweet 
  dict_replies= {} #dict of {created_At_time:[reply tweet]} for every source tweet
  list_replies = []
  dict_replies_list = []
  for line in train_data_file:
    train_Data_list = json.loads(line)
    df = pd.DataFrame.from_dict(train_Data_list)
    # df = df[['screen_name', 'user_id', 'followers_count', 'friends_count', 'statuses_count', 'listed_count', 'favourites_count', 'verified', 'location', 'time_zone', 'geo_enabled', 'user_created_at', 'protected', 'default_profile', 'id', 'in_reply_to_status_id_str', 'text', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'favorited', 'favorite_count', 'retweeted', 'retweet_count', 'created_at', 'label']]
    # df.rename(columns={'id_str': 'id'}, inplace=True)
    df.rename(columns={'text': 'tweet'}, inplace=True)
    replies_list,len_reply_list,dict_replies = get_replies(df) #reply list for a single source tweet
    replies_list_2d = list(dict_replies.values()) #get all the replies for a single source tweet
    # print(len(replies_list_2d))
    replies_list_1d = [j for sub in replies_list_2d for j in sub]
    # print(len(replies_list_1d))
    reply_length = len(replies_list_1d)
    list_replies.append(reply_length) 
    dict_replies_list.append(dict_replies)

    chunks.append(df)
    list_of_rlists.append(replies_list)
    num_replies.append(len_reply_list)
    # print("The output of list_of_rlists in every iteration=",list_of_rlists,"\n",len(list_of_rlists))
  dfs1 = pd.concat(chunks)
train_data_file.close()
#print(dfs1)
# print("The len of list of list of replies=", len(list_of_rlists))


with open("train.label.json") as train_label_file:
    train_label_json_file = json.load(train_label_file)
    df_train_label = pd.DataFrame(list(train_label_json_file.items()),columns = ['id', 'label'])
train_label_file.close()

#check for class imbalance
# print(df_train_label)
# print(df_train_label.groupby('label').size())
# non - rumour
# 3058
# rumour
# 1583a

#check the id and if it is present in rumours/non-rumours labels, then add it to the row of that dataframe (check from the labels dataframe)
df_final1 = pd.merge(dfs1, df_train_label, how='left', left_on='id_str', right_on='id')
df_final2 = pd.merge(dfs1, df_train_label, how='inner', left_on='id_str', right_on='id')

# print(df_final1)
print(df_final1.shape) #(81120, 7) (81120, 29)

# print(df_final2)
print(df_final2.shape) #(4641, 7) (4641, 29)


#only text and label - drop columns from old one and store in new one 
# Corpus = df_final2
Corpus = df_final2[["id_str","tweet", "label"]]
Corpus.rename(columns={'label': 'labels'}, inplace=True)
Corpus.rename(columns={'id_str': 'id'}, inplace=True)
le = preprocessing.LabelEncoder()

reply_dict_list = get_sorted_by_time(dict_replies_list) #list of dicts 
sorted_replies = []
for i in range(len(reply_dict_list)):
  values_2d = reply_dict_list[i].values()
  values_list_1d = [j for sub in values_2d for j in sub ]
  sorted_replies.append(values_list_1d)

Corpus['label'] = le.fit_transform(Corpus['labels'])
Corpus["all_replies"] = list_of_rlists #has a list of replies 
Corpus["num_replies"] = num_replies
Corpus["dict_replies"] = list_replies
Corpus["all_replies_by_time"] = sorted_replies
print(Corpus.head(10))
print(Corpus.dtypes)
print(Corpus.shape)

source_train = pd.DataFrame()
source_train = fill_df(source_train,df_final2)
source_train["labels"] = df_final2["label"]
source_train["all_replies"] = Corpus["all_replies"] #has a list of replies 
source_train["num_replies"] = Corpus["num_replies"]
source_train["all_replies_by_time"] = Corpus["dict_replies"]
source_train["dict_replies"] = Corpus["all_replies_by_time"]
print(source_train.shape)
print(source_train.columns)

all_train = pd.DataFrame()
all_train = fill_df(all_train,df_final1)
all_train["labels"] = df_final1["label"]
print(all_train.shape)
print(all_train.columns)

### 2.2. Load Development/Validation Data
The train.data.jsonl has about 4k source tweets and about 81k source + reply tweets.

The label files ([train,dev].label.json), are standard JSON files:
{
"552800070199148544": "non-rumour",
"544388259359387648": "non-rumour",
"552805970536333314": "non-rumour",
"525071376084791297": "rumour"
} 

All data files ([train,dev,test,covid].data.jsonl) are JSONL files, where each line is a JSON string. For these files, each line is an event: a list of tweets where the first tweet is a source tweet and the rest are reply tweets.

Here the train data is loaded and merged with the labels and the resulting columns are "id_str","in_reply_to_status_id_str","text", "user","favorited","in_reply_to_user_id_str","created_at", "label"].

In [ ]:
with open("dev.data.jsonl", 'r', encoding='utf-8') as dev_data_file:
  list_of_rlists = []
  chunks = []
  num_replies = []
  dict_replies= {}
  list_replies = []
  dict_replies_list = []
  for line in dev_data_file:
    dev_Data_list = json.loads(line)
    df = pd.DataFrame.from_dict(dev_Data_list)
    # df = df[["id_str","in_reply_to_status_id_str","text", "user","favorited","in_reply_to_user_id_str","created_at"]]
    # df.rename(columns={'id_str': 'id'}, inplace=True)
    df.rename(columns={'text': 'tweet'}, inplace=True)

    # replies_list,len_reply_list = get_replies(df)
    replies_list,len_reply_list,dict_replies = get_replies(df) #reply list for a single source tweet
    replies_list_2d = list(dict_replies.values()) #get all the replies for a single source tweet
    # print(len(replies_list_2d))
    replies_list_1d = [j for sub in replies_list_2d for j in sub]
    # print(len(replies_list_1d))
    reply_length = len(replies_list_1d)
    list_replies.append(reply_length) 
    dict_replies_list.append(dict_replies)

    chunks.append(df)
    num_replies.append(len_reply_list)
    list_of_rlists.append(replies_list)
  dfs_dev = pd.concat(chunks)
dev_data_file.close()
#print(dfs_dev)

with open("dev.label.json") as dev_label_file:
    dev_label_json_file = json.load(dev_label_file)
    df_dev_label = pd.DataFrame(list(dev_label_json_file.items()),columns = ['id', 'label'])
dev_label_file.close()

#check the id and if it is present in rumours/non-rumours labels, then add it to the row of that dataframe (check from the labels dataframe)
df_final1_dev = pd.merge(dfs_dev, df_dev_label, how='left', left_on='id_str', right_on='id')
df_final2_dev = pd.merge(dfs_dev, df_dev_label, how='inner', left_on='id_str', right_on='id')

print(df_final1_dev.shape)
print(df_final2_dev.shape)

reply_dict_list = get_sorted_by_time(dict_replies_list) #list of dicts 
sorted_replies = []
for i in range(len(reply_dict_list)):
  values_2d = reply_dict_list[i].values()
  values_list_1d = [j for sub in values_2d for j in sub ]
  sorted_replies.append(values_list_1d)

#only text and label - drop columns from old one and store in new one 
Corpus_dev = df_final2_dev[["id_str","tweet", "label"]]
Corpus_dev.rename(columns={'label': 'labels'}, inplace=True)
Corpus_dev.rename(columns={'id_str': 'id'}, inplace=True)
# Corpus_dev = df_final2_dev
le = preprocessing.LabelEncoder()
Corpus_dev['label'] = le.fit_transform(Corpus_dev['labels'])
Corpus_dev["all_replies"] = list_of_rlists #has a list of replies 
Corpus_dev["num_replies"] = num_replies
Corpus_dev["all_replies_by_time"] = sorted_replies
Corpus_dev["dict_replies"] = list_replies

print(Corpus_dev.head(10))
print(Corpus_dev.shape)
print(Corpus_dev.dtypes)

source_dev = pd.DataFrame()
source_dev = fill_df(source_dev,df_final2_dev)
source_dev["labels"] = df_final2_dev["label"]
source_dev["all_replies"] = list_of_rlists #has a list of replies 
source_dev["num_replies"] = num_replies
source_dev["all_replies_by_time"] = sorted_replies
source_dev["dict_replies"] = list_replies
print(source_dev.shape)
print(source_dev.columns)

all_dev = pd.DataFrame()
all_dev = fill_df(all_dev,df_final1_dev)
all_dev["labels"] = df_final1_dev["label"]
print(all_dev.shape)
print(all_dev.columns)


In [ ]:

diff =Corpus_dev[Corpus_dev.num_replies != Corpus_dev.dict_replies]
print(len(diff))
diff
print(diff)

### 2.3. Load Test Data
The test data contains 4555 examples with no label. About 300 examples are non-complaining tweets. Our task is to identify their `id` and examine manually whether our results are correct.

In [ ]:
with open("test.data.jsonl", 'r', encoding='utf-8') as test_data_file:
  list_of_rlists = []
  chunks = []
  num_replies = []
  dict_replies= {}
  list_replies = []
  dict_replies_list = []

  for line in test_data_file:
    train_Data_list = json.loads(line)
    df = pd.DataFrame.from_dict(train_Data_list)
    # df = df[["id_str","in_reply_to_status_id_str","text", "user","favorited","in_reply_to_user_id_str","created_at"]]
    # df.rename(columns={'id_str': 'id'}, inplace=True)
    df.rename(columns={'text': 'tweet'}, inplace=True)
    # replies_list,len_reply_list = get_replies(df)
    
    replies_list,len_reply_list,dict_replies = get_replies(df) #reply list for a single source tweet
    replies_list_2d = list(dict_replies.values()) #get all the replies for a single source tweet
    # print(len(replies_list_2d))
    replies_list_1d = [j for sub in replies_list_2d for j in sub]
    # print(len(replies_list_1d))
    reply_length = len(replies_list_1d)
    list_replies.append(reply_length) 
    dict_replies_list.append(dict_replies)

    chunks.append(df)
    num_replies.append(len_reply_list)
    list_of_rlists.append(replies_list)
  dfs1 = pd.concat(chunks)
test_data_file.close()
#print(dfs1)
finalll = dfs1[dfs1['in_reply_to_status_id_str'].isna()] #source tweets
# print(finalll)

reply_dict_list = get_sorted_by_time(dict_replies_list) #list of dicts 
sorted_replies = []
for i in range(len(reply_dict_list)):
  values_2d = reply_dict_list[i].values()
  values_list_1d = [j for sub in values_2d for j in sub ]
  sorted_replies.append(values_list_1d)

#only text - drop columns from old one and store in new one 
Corpus_test = finalll[["id_str","tweet"]]
# Corpus_test = finalll
Corpus_test.rename(columns={'id_str': 'id'}, inplace=True)
Corpus_test["all_replies"] = list_of_rlists #has a list of replies 
Corpus_test["num_replies"] = num_replies
Corpus_test["dict_replies"] = list_replies
Corpus_test["all_replies_by_time"] = sorted_replies

print(Corpus_test.head(10))
print(Corpus_test.shape)
print(Corpus_test.dtypes)

source_test = pd.DataFrame()
source_test = fill_df(source_test,finalll)
source_test["all_replies"] = list_of_rlists #has a list of replies 
source_test["num_replies"] = num_replies
source_test["dict_replies"] = list_replies
source_test["all_replies_by_time"] = sorted_replies
print(source_test.shape)
print(source_test.columns)

all_test = pd.DataFrame()
all_test = fill_df(all_test,dfs1)
print(all_test.shape)
print(all_test.columns)

In [ ]:
diff =Corpus_test[Corpus_test.num_replies != Corpus_test.dict_replies]
print(Corpus_test.iloc[0]["tweet"])
print(Corpus_test.iloc[0]["all_replies_by_time"])
print(Corpus_dev.iloc[0]["tweet"])
print(Corpus_dev.iloc[0]["all_replies_by_time"])
print(len(diff))
diff
print(diff)

## Setting input format/sentence structure for BERT

In [ ]:
replies_train = [' [SEP] '.join([str(tweet) for tweet in all_replies]) for all_replies in Corpus.all_replies_by_time.values]
replies_dev = [' [SEP] '.join([str(tweet) for tweet in all_replies]) for all_replies in Corpus_dev.all_replies_by_time.values]
replies_test = [' [SEP] '.join([str(tweet) for tweet in all_replies]) for all_replies in Corpus_test.all_replies_by_time.values]
X_train = list(zip(Corpus.tweet.values, replies_train)) #concatenate all replies sorted by time to the source tweets
X_val = list(zip(Corpus_dev.tweet.values, replies_dev))
y_train = Corpus.label.values
y_val = Corpus_dev.label.values
X_test = list(zip(Corpus_test.tweet.values, replies_test))
print(len(X_train),len(X_val),len(X_test))
print(type(X_train))
for i in range(10):
  print(X_val[i])
  print(X_val[i][0],"\n") #printing only the source tweets 

## Set up GPU for training

Google Colab offers free GPUs and TPUs. Since we'll be training a large neural network it's best to utilize these features.

A GPU can be added by going to the menu and selecting:

`Runtime -> Change runtime type -> Hardware accelerator: GPU`

Then we need to run the following cell to specify the GPU as the device.

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    # print("preds", preds)
    fpr, tpr, threshold = roc_curve(y_true, preds)
    # print(fpr, tpr, threshold)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

# Step 3 - Fine-tuning BERT

## 1. Install the Hugging Face Library

The transformer library of Hugging Face contains PyTorch implementation of state-of-the-art NLP models including BERT (from Google), GPT (from OpenAI) ... and pre-trained model weights.

In [ ]:
!pip install transformers==2.8.0

## 2. Tokenization and Input Formatting

Before tokenizing our text, we will perform some slight processing on our text including removing entity mentions (eg. @united) and some special character. The level of processing here is much less than in previous approachs because BERT was trained with the entire sentences.

In [ ]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text_ = [re.sub(r'(@.*?)[\s]', ' ', i) for i in text]

    # Replace '&amp;' with '&'
    text_ = [re.sub(r'&amp;', '&', i) for i in text_]

    # Remove trailing whitespace
    text_ = [re.sub(r'\s+', ' ', i).strip() for i in text_]
    text_ = ' [CLS] ' + text_[0] + ' [SEP] ' + text_[1] + ' [SEP] '
    return text_

In [ ]:
# Print sentence 0
print('Original: ', X_train[1])
# for i in range(10):
#     print(X_train[i])
print('Processed: ', text_preprocessing(X_train[0]))

### 2.1. BERT Tokenizer

In order to apply the pre-trained BERT, we must use the tokenizer provided by the library. This is because (1) the model has a specific, fixed vocabulary and (2) the BERT tokenizer has a particular way of handling out-of-vocabulary words.

In addition, we are required to add special tokens to the start and end of each sentence, pad & truncate all sentences to a single constant length, and explicitly specify what are padding tokens with the "attention mask".

The `encode_plus` method of BERT tokenizer will:

(1) split our text into tokens,

(2) add the special `[CLS]` and `[SEP]` tokens, and

(3) convert these tokens into indexes of the tokenizer vocabulary,

(4) pad or truncate sentences to max length, and

(5) create attention mask.






In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
      
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
#         Set MAX_LEN = 512

# Pad input_structure to MAX_LEN
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

Before tokenizing, we need to specify the maximum length of our sentences.







Now let's tokenize our data.

In [ ]:
# Specify `MAX_LEN`
# MAX_LEN = 64
MAX_LEN = 512

# Print sentence 0 and its encoded token ids
# token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
# print('Original: ', X[0])
# print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
# print(X_train, type(X_train), len(X_train))
# print(X_val, type(X_val), len(X_val))

train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)
test_inputs, test_masks = preprocessing_for_bert(X_test)

print(train_inputs, train_masks, val_inputs, val_masks)


### 2.2. Create PyTorch DataLoader

We will create an iterator for our dataset using the torch DataLoader class. This will help save on memory during training and boost the training speed.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)

Corpus_dev['label'] = le.fit_transform(Corpus_dev['label'])
y_val=Corpus_dev.label.values
# print(type(y_val))
# print(y_val)
# print(Corpus_dev.dtypes)
# print(Corpus_dev.head())
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 10

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
print(train_data,train_sampler,train_dataloader)
# Create the DataLoader for our validation set

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 3. Train Our Model

### 3.1. Create BertClassifier

BERT-base consists of 12 transformer layers, each transformer layer takes in a list of token embeddings, and produces the same number of embeddings with the same hidden size (or dimensions) on the output. The output of the final transformer layer of the `[CLS]` token is used as the features of the sequence to feed a classifier.

The `transformers` library has the [`BertForSequenceClassification`](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#bertforsequenceclassification) class which is designed for classification tasks. However, we will create a new class so we can specify our own choice of classifiers.

Below we will create a BertClassifier class with a BERT model to extract the last hidden layer of the `[CLS]` token and a single-hidden-layer feed-forward neural network as our classifier.

In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

### 3.2. Optimizer & Learning Rate Scheduler

To fine-tune our Bert Classifier, we need to create an optimizer. The authors recommend following hyper-parameters:

- Batch size: 16 or 32
- Learning rate (Adam): 5e-5, 3e-5 or 2e-5
- Number of epochs: 2, 3, 4

Huggingface provided the [run_glue.py](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L109) script, an examples of implementing the `transformers` library. In the script, the AdamW optimizer is used.

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=2):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

### 3.3. Training Loop

We will train our Bert Classifier for 4 epochs. In each epoch, we will train our model and evaluate its performance on the validation set. In more details, we will:

Training:
- Unpack our data from the dataloader and load the data onto the GPU
- Zero out gradients calculated in the previous pass
- Perform a forward pass to compute logits and loss
- Perform a backward pass to compute gradients (`loss.backward()`)
- Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
- Update the model's parameters (`optimizer.step()`)
- Update the learning rate (`scheduler.step()`)

Evaluation:
- Unpack our data and load onto the GPU
- Forward pass
- Compute loss and accuracy rate over the validation set

The script below is commented with the details of our training and evaluation loop. 

In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=2, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()
                torch.save(model.state_dict(), f'/content/drive/MyDrive/NLP_data/finetuned_BERT_epoch_{epoch_i}_seed22_new.model')

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)
    
        print("-"*70)
        
        torch.save(model.state_dict(), f'/content/drive/MyDrive/NLP_data/finetuned_BERT_epoch_{epochs}_seed22_new.model')

        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
  import torch,gc 
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

Now, let's start training our BertClassifier!

In [ ]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
# train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

# Step 4 - Load model for prediction and evaluation



In [ ]:
model1_seed42_path = "finetuned_BERT_epoch_2_seed42_new.model"
model1_seed42  = BertClassifier()
model1_seed42.load_state_dict(torch.load(model1_seed42_path, map_location=torch.device('cuda')))
model1_seed42.to(device)
model1_seed42.eval()

model1_seed22_path = "finetuned_BERT_epoch_2_seed22_new.model"
model1_seed22  = BertClassifier()
model1_seed22.load_state_dict(torch.load(model1_seed22_path, map_location=torch.device('cuda')))
model1_seed22.to(device)
model1_seed22.eval()

model1_seed82_path = "finetuned_BERT_epoch_2_seed82_new.model"
model1_seed82  = BertClassifier()
model1_seed82.load_state_dict(torch.load(model1_seed82_path, map_location=torch.device('cuda')))
model1_seed82.to(device)
model1_seed82.eval()

device = torch.device("cuda")
model1_seed44_path = "finetuned_BERT_epoch_2_seed44_new.model"
model1_seed44  = BertClassifier()
model1_seed44.load_state_dict(torch.load(model1_seed44_path, map_location=torch.device('cuda')))
model1_seed44.to(device)
model1_seed44.eval()

model1_seed46_path = "finetuned_BERT_epoch_2_seed46_new.model"
model1_seed46  = BertClassifier()
model1_seed46.load_state_dict(torch.load(model1_seed46_path, map_location=torch.device('cuda')))
model1_seed46.to(device)
model1_seed46.eval()

model1_seed26_path = "finetuned_BERT_epoch_2_seed26_new.model"
model1_seed26  = BertClassifier()
model1_seed26.load_state_dict(torch.load(model1_seed26_path, map_location=torch.device('cuda')))
model1_seed26.to(device)
model1_seed26.eval()

model1_seed52_path = "finetuned_BERT_epoch_2_seed52_new.model"
model1_seed52  = BertClassifier()
model1_seed52.load_state_dict(torch.load(model1_seed52_path, map_location=torch.device('cuda')))
model1_seed52.to(device)
model1_seed52.eval()

model1_seed78_path = "finetuned_BERT_epoch_2_seed78_new.model"
model1_seed78  = BertClassifier()
model1_seed78.load_state_dict(torch.load(model1_seed78_path, map_location=torch.device('cuda')))
model1_seed78.to(device)
model1_seed78.eval()

model1_seed86_path = "finetuned_BERT_epoch_2_seed86_new.model"
model1_seed86  = BertClassifier()
model1_seed86.load_state_dict(torch.load(model1_seed86_path, map_location=torch.device('cuda')))
model1_seed86.to(device)
model1_seed86.eval()

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.

    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)
    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

def get_avg_preds(ensemble_predds, Temp):
    ensemble_list = []
    for pred in ensemble_predds:
        pred_0_list = []
        pred_1_list = []

        # new_confidences = new_conficence_vals(pred, Temp)
        # softmax_preds = get_softmax_preds(new_confidences)
        
        for i in pred:
            pred_0_list.append(i[0])
            pred_1_list.append(i[1])
            # print(i,i[0],i[1])
        pred_0_avg = sum(pred_0_list) / len(pred_0_list)
        pred_1_avg = sum(pred_1_list) / len(pred_1_list)
        
        ensemble_list.append([pred_0_avg, pred_1_avg])

    return ensemble_list

### 4.1. Evaluation on Validation Set



In [ ]:
# Compute predicted probabilities on the val/test set
probs1_val = bert_predict(model1_seed44, val_dataloader)
probs2_val = bert_predict(model1_seed46, val_dataloader)
probs3_val = bert_predict(model1_seed26, val_dataloader)
probs4_val = bert_predict(model1_seed52, val_dataloader)
probs5_val = bert_predict(model1_seed42, val_dataloader)
probs6_val = bert_predict(model1_seed22, val_dataloader)
probs7_val = bert_predict(model1_seed82, val_dataloader)
probs8_val = bert_predict(model1_seed78, val_dataloader)

# print(all_logits1, all_logits2, all_logits3) #tensor([[-1.5097,  2.0246] to [0.02835082 0.97164917]
# print(probs1, probs2, probs3)


In [ ]:
probs9_val = bert_predict(model1_seed86, val_dataloader)


In [ ]:
# ensemble_predictions_zip = tuple(zip(probs1, probs2, probs3, probs4, probs5))
# ensemble_predictions_zip_val = tuple(zip(probs1_val, probs2_val, probs3_val, probs4_val, probs5_val, probs6_val, probs7_val,probs8_val,probs9_val))
ensemble_predictions_zip_val = tuple(zip(probs2_val,probs7_val))

ensemble_predictions_val = get_avg_preds(ensemble_predictions_zip_val,1)

probs_final_val = np.array(ensemble_predictions_val) #convert list of lists into numpy.ndarray
evaluate_roc(probs_final_val, y_val)

threshold_val = 0.45
# Get predictions from the probabilities
preds_val = np.where(probs_final_val[:, 1] > threshold_val, 1, 0)
# print(preds)
print(len(preds_val))
print("Number of tweets predicted rumour: ", preds_val.sum())

unique_val, counts_val = np.unique(preds_val, return_counts=True)
print(dict(zip(unique_val, counts_val)))

Corpus_pred_val = Corpus_dev
# print(Corpus_pred_val.head())
# y_predicted = nb_model.predict(X_test_tfidf)
y_labels = ["non-rumour" if i==0 else "rumour" for i in preds_val]
# print(y_predicted,y_predicted_labels)
Corpus_pred_val['label'] = y_labels
# print(Corpus_pred_val.head())
# Corpus_pred_val.set_index('id',inplace=True)
dictt_val = Corpus_pred_val.to_dict()['label']
print(dictt_val)
print(len(dictt_val))

with open("dev.baseline_bertFen.json", "w") as outfile: 
    json.dump(dictt_val, outfile)
outfile.close()

#using the eval.py code to print precision, recall and f1 score
import argparse
import sys
import json
from sklearn.metrics import precision_recall_fscore_support

#parameters
debug = False

def convert_label(label):
    if label == "rumour":
        return 1
    elif label == "non-rumour":
        return 0
    else:
        raise Exception("label classes must be 'rumour' or 'non-rumour'")

# groundtruth = json.load(open(args.groundtruth))
# predictions = json.load(open(args.predictions))
with open("dev.label.json") as infile1: 
    groundtruth = json.load(infile1)
infile1.close()
with open("dev.baseline_bertFen.json") as infile2: 
    predictions = json.load(infile2)
infile2.close()

y_true, y_pred = [], []
def perf_measure(y_actual, y_hat, validation_set_ids):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    fp_ids = []
    tp_ids = []
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
           tp_ids.append(validation_set_ids[i])
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
           fp_ids.append(validation_set_ids[i])
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return TP, FP, TN, FN, tp_ids, fp_ids
try:
    for k, v in groundtruth.items():
        if k in predictions:
            y_pred.append(convert_label(predictions[k]))
        #if ID isn't in predictions, assume incorrect predictions
        else:
            y_pred.append(int(not(bool(convert_label(v)))))
        y_true.append(convert_label(v))
    prediction_ids = list(predictions.keys())
    TP, FP, TN, FN, tp_ids,fp_ids = perf_measure(y_true, y_pred, prediction_ids) #code for error analysis
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, pos_label=1, average="binary")
except Exception as error:
    print("Error:", error)
    raise SystemExit
print("TP, FP, TN, FN, fp_ids \n", TP, FP, TN, FN, tp_ids,fp_ids)
print("Performance on the rumour class:")
print("Precision =", p)
print("Recall    =", r)
print("F1        =", f)

In [ ]:
tp_filter_df = Corpus_dev[Corpus_dev.index.isin(tp_ids)]
print(tp_filter_df.shape)
tp_filter_df['tweet'].tolist()

In [ ]:
#Get the false positive tweets from the ids 
fp_filter_df = Corpus_dev[Corpus_dev.index.isin(fp_ids)]
print(fp_filter_df.shape)
fp_filter_df['tweet'].tolist()

## 4.2. Predictions on Test Set

In [ ]:
# Compute predicted probabilities on the test set
probs1 = bert_predict(model1_seed44, test_dataloader)
probs2 = bert_predict(model1_seed46, test_dataloader)
probs3 = bert_predict(model1_seed26, test_dataloader)
probs4 = bert_predict(model1_seed52, test_dataloader)
probs5 = bert_predict(model1_seed42, test_dataloader)
probs6 = bert_predict(model1_seed22, test_dataloader)
probs7 = bert_predict(model1_seed82, test_dataloader)
probs8 = bert_predict(model1_seed78, test_dataloader)
probs9 = bert_predict(model1_seed86, test_dataloader)

# ensemble_predictions_zip = tuple(zip(probs1, probs2, probs3, probs4,probs5))


In [ ]:
# ensemble_predictions_zip = tuple(zip(probs1, probs2, probs3, probs4, probs5, probs6, probs7))
ensemble_predictions_zip = tuple(zip(probs2, probs7))
# ensemble_predictions_zip_val = tuple(zip(probs1_val, probs2_val, probs3_val, probs4_val, probs5_val, probs6_val, probs7_val,probs8_val,probs9_val))

ensemble_predictions = get_avg_preds(ensemble_predictions_zip,1)
probs_final = np.array(ensemble_predictions) #convert list of lists into numpy.ndarray
# print(probs_final)
print(len(probs_final))
# Get predictions from the probabilities
threshold = 0.45
preds = np.where(probs_final[:, 1] > threshold, 1, 0)
# print(preds)
print(len(preds))

# Number of tweets predicted non-negative
print("Number of tweets predicted rumour: ", preds.sum())
# label
# non-rumour    393
# rumour        187

In [ ]:
unique, counts = np.unique(preds, return_counts=True)
dict(zip(unique, counts))

In [ ]:
Corpus_pred_bert = Corpus_test
print(Corpus_pred_bert.head())
# y_predicted = nb_model.predict(X_test_tfidf)
y_labels = ["non-rumour" if i==0 else "rumour" for i in preds]
# print(y_predicted,y_predicted_labels)
Corpus_pred_bert['label'] = y_labels
print(Corpus_pred_bert.head())
# Corpus_pred_bert.set_index('id',inplace=True)
dictt_bert = Corpus_pred_bert.to_dict()['label']
print(dictt_bert)
print(len(dictt_bert))

In [ ]:
with open("test.baseline_bertFen.json", "w") as outfile: 
    json.dump(dictt_bert, outfile)

## 4.3. Covid-19 Test set predictions

In [ ]:
with open("covid.data.jsonl", 'r', encoding='utf-8') as test_data_file:
  list_of_rlists = []
  chunks = []
  num_replies = []
  dict_replies= {}
  list_replies = []
  dict_replies_list = []

  for line in test_data_file:
    train_Data_list = json.loads(line)
    df = pd.DataFrame.from_dict(train_Data_list)
    # df = df[["id_str","in_reply_to_status_id_str","text", "user","favorited","in_reply_to_user_id_str","created_at"]]
    # df.rename(columns={'id_str': 'id'}, inplace=True)
    df.rename(columns={'text': 'tweet'}, inplace=True)
    # replies_list,len_reply_list = get_replies(df)
    
    replies_list,len_reply_list,dict_replies = get_replies(df) #reply list for a single source tweet
    replies_list_2d = list(dict_replies.values()) #get all the replies for a single source tweet
    # print(len(replies_list_2d))
    replies_list_1d = [j for sub in replies_list_2d for j in sub]
    # print(len(replies_list_1d))
    reply_length = len(replies_list_1d)
    list_replies.append(reply_length) 
    dict_replies_list.append(dict_replies)

    chunks.append(df)
    num_replies.append(len_reply_list)
    list_of_rlists.append(replies_list)
  dfs1 = pd.concat(chunks)
test_data_file.close()
#print(dfs1)
finalll = dfs1[dfs1['in_reply_to_status_id_str'].isna()] #source tweets
# print(finalll)

reply_dict_list = get_sorted_by_time(dict_replies_list) #list of dicts 
sorted_replies = []
for i in range(len(reply_dict_list)):
  values_2d = reply_dict_list[i].values()
  values_list_1d = [j for sub in values_2d for j in sub ]
  sorted_replies.append(values_list_1d)

#only text - drop columns from old one and store in new one 
Corpus_test = finalll[["id_str","tweet"]]
# Corpus_test = finalll
Corpus_test.rename(columns={'id_str': 'id'}, inplace=True)
Corpus_test["all_replies"] = list_of_rlists #has a list of replies 
Corpus_test["num_replies"] = num_replies
Corpus_test["dict_replies"] = list_replies
Corpus_test["all_replies_by_time"] = sorted_replies

print(Corpus_test.head(10))
print(Corpus_test.shape)
print(Corpus_test.dtypes)

source_test = pd.DataFrame()
source_test = fill_df(source_test,finalll)
source_test["all_replies"] = list_of_rlists #has a list of replies 
source_test["num_replies"] = num_replies
source_test["dict_replies"] = list_replies
source_test["all_replies_by_time"] = sorted_replies
print(source_test.shape)
print(source_test.columns)

all_test = pd.DataFrame()
all_test = fill_df(all_test,dfs1)
print(all_test.shape)
print(all_test.columns)

In [ ]:
replies_test = [' [SEP] '.join([str(tweet) for tweet in all_replies]) for all_replies in Corpus_test.all_replies_by_time.values]
X_test = list(zip(Corpus_test.tweet.values, replies_test))

# Load test data
test_data = Corpus_test
# Display 5 samples from the test data
test_data.sample(5)

test_inputs, test_masks = preprocessing_for_bert(X_test)

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

probs1 = bert_predict(model1_seed44, test_dataloader)
probs2 = bert_predict(model1_seed46, test_dataloader)
probs3 = bert_predict(model1_seed26, test_dataloader)
probs4 = bert_predict(model1_seed52, test_dataloader)
probs5 = bert_predict(model1_seed42, test_dataloader)
probs6 = bert_predict(model1_seed22, test_dataloader)
probs7 = bert_predict(model1_seed82, test_dataloader)
probs8 = bert_predict(model1_seed78, test_dataloader)
probs9 = bert_predict(model1_seed86, test_dataloader)

ensemble_predictions_zip = tuple(zip(probs2, probs7))

ensemble_predictions = get_avg_preds(ensemble_predictions_zip,1)
probs_final = np.array(ensemble_predictions) #convert list of lists into numpy.ndarray
print(len(probs_final))
# Get predictions from the probabilities
threshold = 0.45
preds = np.where(probs_final[:, 1] > threshold, 1, 0)
# print(preds)
print(len(preds))

# Number of tweets predicted non-negative
print("Number of tweets predicted rumour: ", preds.sum())


unique, counts = np.unique(preds, return_counts=True)
dict(zip(unique, counts))

Corpus_pred_bert = Corpus_test
print(Corpus_pred_bert.head())
# y_predicted = nb_model.predict(X_test_tfidf)
y_labels = ["non-rumour" if i==0 else "rumour" for i in preds]
# print(y_predicted,y_predicted_labels)
Corpus_pred_bert['label'] = y_labels
print(Corpus_pred_bert.head())
# Corpus_pred_bert.set_index('id',inplace=True)
dictt_bert = Corpus_pred_bert.to_dict()['label']
print(dictt_bert)
print(len(dictt_bert))

with open("test.baseline_bertC.json", "w") as outfile: 
    json.dump(dictt_bert, outfile)

In [ ]:
source_dev["label"] = y_labels
df_rumour = source_dev[source_dev['label'] == 'rumour']
df_non_rumour = source_dev[source_dev['label'] == 'non-rumour']
print(df_rumour.shape, df_non_rumour.shape)

In [ ]:
df_rumour.head()

In [ ]:
df_non_rumour.head()

Finding the min and max dates

In [ ]:
from datetime import datetime
new_datetime_list =[]
for index, row in source_test.iterrows():
    new_datetime = datetime.strftime(datetime.strptime(row["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    new_datetime_list.append(new_datetime)

In [ ]:
new_dates= []
for date_time_str in new_datetime_list:
    date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    new_dates.append(date_time_obj.date())
    
oldest = min(new_dates)
youngest = max(new_dates)
print("The tweets started on: ",oldest, "and are extracted until: ",youngest, "which is a period of", youngest-oldest)

In [ ]:
df_non_rumour.head()

# **Question 0 - Figure 1 - number vs time of tweets plot - Basic analysis**

## Account Age Calculation
*From time created until current time*

In [ ]:
def date_diff_in_Seconds(dt2, dt1):
  timedelta = dt2 - dt1
  return timedelta.days * 24 * 3600 + timedelta.seconds

def setAge(df):
    #convert the raw date time from twitter into a string
    user_datetime_list =[]
    for index, row in df.iterrows():
        new_datetime = datetime.strftime(datetime.strptime(row["user_created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        user_datetime_list.append(new_datetime)
    account_age_days_list = []
    account_age_seconds_list = []

    for date_time_str in user_datetime_list:
        date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
        age = datetime.utcnow() - date_time_obj #3007 days 18:09:38.671760
        age_seconds = date_diff_in_Seconds(datetime.utcnow(), date_time_obj)
        account_age_days_list.append(age)
        account_age_seconds_list.append(age_seconds)

    df["account age (days)"] = account_age_days_list
    df["account age (seconds)"] = account_age_seconds_list

def setDate(df):
    #convert the raw date time from twitter into a string
    user_datetime_list =[]
    for index, row in df.iterrows():
        new_datetime = datetime.strftime(datetime.strptime(row["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        user_datetime_list.append(new_datetime)
    user_datetimes= []
    for date_time_str in user_datetime_list:
        date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
        user_datetimes.append(date_time_obj)
    df["Date"]=user_datetimes

setAge(df_rumour)
setAge(df_non_rumour)
setDate(df_rumour)
setDate(df_non_rumour)

In [ ]:
#https://stackoverflow.com/questions/63102989/calculate-tweets-per-day
def tweets_per_day(df):
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
    return df[['tweet']].groupby(df['Date'].dt.date).count()

In [ ]:
tweets_per_day_rumour = tweets_per_day(df_non_rumour)
tweets_per_day_non_rumour = tweets_per_day(df_rumour)
df.sort_index(inplace=True)

In [ ]:
import matplotlib.dates as mdates


In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

x_values = tweets_per_day_non_rumour.index
y_values = tweets_per_day_non_rumour["tweet"]
ax = plt.gca()
# get axes
formatter = mdates.DateFormatter("%Y-%m-%d")
print(formatter)
# format as dates
ax.xaxis.set_major_formatter(formatter)
locator = mdates.DayLocator()
# set locator
ax.xaxis.set_major_locator(locator)
plt.plot(x_values, y_values)

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(12, 12))

# Add x-axis and y-axis
ax.bar(tweets_per_day_non_rumour.index.values,
       tweets_per_day_non_rumour['tweet'],
       color='purple')

# Set title and labels for axes
ax.set(xlabel="Date of NON_Rumours",
       ylabel="Number of NON-Rumour Tweets",
       title="Daily NON-Rumour Tweets\nFeb - Aug 2020 for Covid-19")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

x_values = tweets_per_day_rumour.index
y_values = tweets_per_day_rumour["tweet"]
ax = plt.gca()
# get axes
formatter = mdates.DateFormatter("%Y-%m-%d")
print(formatter)
# format as dates
ax.xaxis.set_major_formatter(formatter)
locator = mdates.DayLocator()
# set locator
ax.xaxis.set_major_locator(locator)
plt.plot(x_values, y_values)

In [ ]:
# Create figure and plot space
fig, ax = plt.subplots(figsize=(12, 12))

# Add x-axis and y-axis
ax.bar(tweets_per_day_rumour.index.values,
       tweets_per_day_rumour['tweet'],
       color='purple')

# Set title and labels for axes
ax.set(xlabel="Date of rumour tweets",
       ylabel="Number of rumour tweets",
       title="Daily Rumour Tweets\nFeb - Aug 2020 for Covid-19")

plt.show()

# **Question 1 - Popular Hashtags**

## Hashtags Method 1

Question 1 - What are the characteristics of rumour-creating users, and how do they differ compared to normal users?

In [ ]:
# Hashtags & mentions
#What are the popular hashtags of COVID-19 rumours and non-rumours? How much overlap or difference do they share?
def get_popular_hashtags(df,n):
    tag_dict = {}
    df_list = df['tweet'].values.tolist()
#     print (df_list)
    for tweet_text in df_list:
#         tweet_text = df.loc[i]['text']
#         print(tweet_text)
        tweet_text_lowercase = tweet_text.lower()
        tweet_tokens = tweet_text_lowercase.split()
        for word in tweet_tokens:
            # Hashtags - tokenize and build dict of tag counts
            if (word[0:1] == '#' and len(word) > 1):
                key = word.translate(str.maketrans("","", string.punctuation))
                if key in tag_dict:
                    tag_dict[key] += 1
                else:
                    tag_dict[key] = 1
    # The 10 most popular tags and counts
    top_tags = dict(sorted(tag_dict.items(), key=operator.itemgetter(1), reverse=True)[:n])
    top_tags_sorted = sorted(top_tags.items(), key=lambda x: x[1])[::-1]
    print (f'Top {n} hashtags:')
    print ('----------------')
    print(top_tags_sorted)
#     for tag in top_tags_sorted:
#         print (tag[0], '-', str(tag[1]))

print("RUMOUR ")
get_popular_hashtags(df_rumour, 40)
print("\nNON-RUMOUR ")
get_popular_hashtags(df_non_rumour,40)


## Hashtags Method 2

In [ ]:
def find_hashtags(tweet):
      '''This function will extract hashtags'''
      tweet_text_lowercase = tweet.lower()
      return re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', tweet_text_lowercase)

def popular_hashtags(df_non_rumour, n):
    df_non_rumour['hashtags'] = df_non_rumour.tweet.apply(find_hashtags)
    # print(df_non_rumour.head())

    # take the rows from the hashtag columns where there are actually hashtags
    hashtags_list_df = df_non_rumour.loc[
                          df_non_rumour.hashtags.apply(
                              lambda hashtags_list: hashtags_list !=[]
                          ),['hashtags']]
    # print(hashtags_list_df.head())

    #To see which hashtags were popular we will need to flatten out this dataframe.
    # create dataframe where each use of hashtag gets its own row
    flattened_hashtags_df = pd.DataFrame(
        [hashtag for hashtags_list in hashtags_list_df.hashtags
        for hashtag in hashtags_list],
        columns=['hashtag'])
    # print(flattened_hashtags_df.head(10))

    # number of unique hashtags
    flattened_hashtags_df['hashtag'].unique().size
    # count of appearances of each hashtag
    popular_hashtags = flattened_hashtags_df.groupby('hashtag').size()\
                                            .reset_index(name='counts')\
                                            .sort_values('counts', ascending=False)\
                                            .reset_index(drop=True)
        # number of times each hashtag appears - not necessary
    counts = flattened_hashtags_df.groupby(['hashtag']).size()\
                                  .reset_index(name='counts')\
                                  .counts

    # define bins for histogram                              
    my_bins = np.arange(0,counts.max()+2, 5)-0.5

    # plot histogram of tweet counts
    plt.figure()
    plt.hist(counts, bins = my_bins)
    plt.xlabels = np.arange(1,counts.max()+1, 1)
    plt.xlabel('hashtag number of appearances')
    plt.ylabel('frequency')
    plt.yscale('log', nonposy='clip')
    plt.show()
    
    return popular_hashtags[:n]
n= 20
print(f"TOP {n} RUMOUR HASHTAGS \n",popular_hashtags(df_rumour,n))
print(f"\nTOP {n} NON-RUMOUR HASHTAGS \n",popular_hashtags(df_non_rumour, n))

In [ ]:
# number of times each hashtag appears - not necessary
counts = flattened_hashtags_df.groupby(['hashtag']).size()\
                              .reset_index(name='counts')\
                              .counts

# define bins for histogram                              
my_bins = np.arange(0,counts.max()+2, 5)-0.5

# plot histogram of tweet counts
plt.figure()
plt.hist(counts, bins = my_bins)
plt.xlabels = np.arange(1,counts.max()+1, 1)
plt.xlabel('hashtag number of appearances')
plt.ylabel('frequency')
plt.yscale('log', nonposy='clip')
plt.show()

In [ ]:
# take hashtags which appear at least this amount of times
min_appearance = 10
# find popular hashtags - make into python set for efficiency
popular_hashtags_set = set(popular_hashtags[popular_hashtags.counts>=min_appearance]['hashtag'])
# make a new column with only the popular hashtags
hashtags_list_df['popular_hashtags'] = hashtags_list_df.hashtags.apply(lambda hashtag_list: [hashtag for hashtag in hashtag_list if hashtag in popular_hashtags_set])
# drop rows without popular hashtag
popular_hashtags_list_df = hashtags_list_df.loc[hashtags_list_df.popular_hashtags.apply(lambda hashtag_list: hashtag_list !=[])]

# make new dataframe
hashtag_vector_df = popular_hashtags_list_df.loc[:, ['popular_hashtags']]
for hashtag in popular_hashtags_set:
    # make columns to encode presence of hashtags
    hashtag_vector_df['{}'.format(hashtag)] = hashtag_vector_df.popular_hashtags.apply(lambda hashtag_list: int(hashtag in hashtag_list))
    
hashtag_matrix = hashtag_vector_df.drop('popular_hashtags', axis=1)

In [ ]:
# calculate the correlation matrix
correlations = hashtag_matrix.corr(method ='pearson')

# plot the correlation matrix
plt.figure(figsize=(10,10))
sns.heatmap(correlations,
    cmap='RdBu',
    vmin=-1,
    vmax=1,
    square = True,
    cbar_kws={'label':'correlation'})
plt.show()

# **Question 2 - Sentiment Analysis**



In [ ]:
# Reference - https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/analyze-tweet-sentiment-in-python/#:~:text=tweet%20in%20tweets%5D-,Analyze%20Sentiments%20in%20Tweets,polarity%20value%20using%20the%20attribute%20.
def remove_url(txt):
#     Replace URLs found in a text string with nothing (i.e. it will remove the URL from the string).
#     Parameters txt : string A text string that you want to parse and remove urls.
#     Returns The same txt string with url's removed.
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
tweets_no_urls = []
for i in range(len(df_rumour)):
    tweets_no_urls.append(remove_url(df_rumour.iloc[i]['tweet']))

In [ ]:
# Create textblob objects of the tweets
sentiment_objects = [TextBlob(tweet) for tweet in tweets_no_urls]
print(sentiment_objects[0].polarity, sentiment_objects[0])

# Create list of polarity valuesx and tweet text
sentiment_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]
print(sentiment_values[10])

# Create dataframe containing the polarity value and tweet text
sentiment_df = pd.DataFrame(sentiment_values, columns=["polarity", "tweet"])
print(sentiment_df.head())

%matplotlib inline
fig, ax = plt.subplots(figsize=(8, 6))

# Plot histogram of the polarity values
sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="purple")

plt.title("Sentiments from Rumour Tweets on Covid-19")
plt.show()
# Remove polarity values equal to zero
sentiment_df = sentiment_df[sentiment_df.polarity != 0]
fig, ax = plt.subplots(figsize=(8, 6))

# Plot histogram with break at zero
sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="purple")

plt.title("Sentiments from Rumour Tweets on Covid-19")
plt.show()

In [ ]:
all_tweets_no_urls = []
for i in range(len(df_non_rumour)):
    all_tweets_no_urls.append(remove_url(df_non_rumour.iloc[i]['tweet']))
all_tweets_no_urls[:5]

In [ ]:
# Create textblob objects of the tweets
sentiment_objects = [TextBlob(tweet) for tweet in all_tweets_no_urls]
# Calculate polarity of tweets
wild_sent_values = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objects]
# Create dataframe containing polarity values and tweet text
wild_sent_df = pd.DataFrame(wild_sent_values, columns=["polarity", "tweet"])
wild_sent_df = wild_sent_df[wild_sent_df.polarity != 0]
wild_sent_df.head()
fig, ax = plt.subplots(figsize=(8, 6))

wild_sent_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1],
        ax=ax, color="purple")

plt.title("Sentiments from Non Rumour Tweets on Covid-19")
plt.show()
#What does the histogram of the polarity values tell you about sentiments in the tweets gathered from the search “#climate+change -filter:retweets”? Are they more positive or negative?

In [ ]:
def set_sentiment(df1):
    sentiment_list = []
    for index, row in df1.iterrows():
        tweet_text = row['tweet'] #each row has one sentence and get that sentence 
        analysis = TextBlob(tweet_text)    
    #     print(analysis.sentiment)    
        if analysis.sentiment[0]>0:       
            sentiment_list.append('Positive')    
        elif analysis.sentiment[0]<0:       
            sentiment_list.append('Negative')     
        else:       
            sentiment_list.append('Neutral')
    df1['sentiment'] = sentiment_list
    # print(df1.head())
    print(df1.groupby('sentiment').size())
set_sentiment(df_rumour)
set_sentiment(df_non_rumour)

create a datadrame for reply tweets and do the same sentiment analysis as done for source tweets

Calculate Sentiment ratio

In [ ]:
rum_ratio = '259:164:518'
non_rum_ratio = '3812:4814:7892'
def solve(ratio):
    numbers = [int(i) for i in ratio.split(':')]
    denominater = reduce(gcd,numbers)
    solved = [i/denominater for i in numbers]
    return ':'.join(str(i) for i in solved)

rumour_ratio =  solve(rum_ratio)
print (rumour_ratio)
non_rumour_ratio =  solve(non_rum_ratio)
print (non_rumour_ratio)

n1 = 260
n2 = 164
n3 = 518

print("THE GCD OF GIVEN NUMBERS:",math.gcd(math.gcd(n1,n2),n3))

n1 = 3812
n2 = 4814
n3 = 7891

print("THE GCD OF GIVEN NUMBERS:",math.gcd(math.gcd(n1,n2),n3))

# **Question 3 - Topic analysis - Figure 4, Table 5**

In [ ]:
# Reference https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ https://www.youtube.com/watch?v=UEn3xHNBXJU
#https://github.com/bhargavvader/personal/issues/5
#https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0

In [ ]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens
    
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
    
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

## Rumour Topics

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sentence_list = list(df_rumour["tweet"])
text_data = []
for line in sentence_list:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        text_data.append(tokens)
#First, we are creating a dictionary from the data, then convert to bag-of-words corpus and save the dictionary and corpus for future use.
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

#We are asking LDA to find 3/5/10 topics in the data:
# NUM_TOPICS = 5
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# ldamodel.save('model5_rumour.gensim')
# topics = ldamodel.print_topics(num_words=4)
# for topic in topics:
#     print(topic)

# #Now we are asking LDA to find 3 topics in the data:
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
# ldamodel.save('model3_rumour.gensim')
# topics = ldamodel.print_topics(num_words=4)
# for topic in topics:
#     print(topic)

#We can also find 10 topics:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10_rumour.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
# lda = gensim.models.ldamodel.LdaModel.load('model5_rumour.gensim')
# # feed the LDA model into the pyLDAvis instance
# # lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)
# lda_display = gensimvis.prepare(lda, corpus, dictionary, sort_topics=False)
# pyLDAvis.display(lda_display)

# lda3 = gensim.models.ldamodel.LdaModel.load('model3_rumour.gensim')
# lda_display3 = gensimvis.prepare(lda3, corpus, dictionary, sort_topics=False)
# pyLDAvis.display(lda_display3)

lda10 = gensim.models.ldamodel.LdaModel.load('model10_rumour.gensim')
lda_display10 = gensimvis.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

## Non-Rumour Topics

In [ ]:
sentence_list = list(df_non_rumour["tweet"])
text_data = []
for line in sentence_list:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        text_data.append(tokens)

#First, we are creating a dictionary from the data, then convert to bag-of-words corpus and save the dictionary and corpus for future use.
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

# #We are asking LDA to find 5 topics in the data:
# NUM_TOPICS = 5
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# ldamodel.save('model5_non_rumour.gensim')
# topics = ldamodel.print_topics(num_words=4)
# for topic in topics:
#     print(topic)

# #Now we are asking LDA to find 3 topics in the data:
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
# ldamodel.save('model3_non_rumour.gensim')
# topics = ldamodel.print_topics(num_words=4)
# for topic in topics:
#     print(topic)

#We can also find 10 topics:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10_non_rumour.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

# dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
# corpus = pickle.load(open('corpus.pkl', 'rb'))
# lda = gensim.models.ldamodel.LdaModel.load('model5_non_rumour.gensim')
# # feed the LDA model into the pyLDAvis instance
# # lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)
# lda_display = gensimvis.prepare(lda, corpus, dictionary, sort_topics=False)
# pyLDAvis.display(lda_display)

# lda3 = gensim.models.ldamodel.LdaModel.load('model3_non_rumour.gensim')
# lda_display3 = gensimvis.prepare(lda3, corpus, dictionary, sort_topics=False)
# pyLDAvis.display(lda_display3)

lda10 = gensim.models.ldamodel.LdaModel.load('model10_non_rumour.gensim')
lda_display10 = gensimvis.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

## Word Cloud

In [ ]:
#https://stackoverflow.com/questions/62525780/how-to-create-wordcloud-showing-most-common-bigrams-in-a-text-using-python
def remove_url(txt):
    """Replace URLs found in a text string with nothing (i.e. it will remove the URL from the string).
    Parameters= txt : string A text string that you want to parse and remove urls.
    Returns=The same txt string with url's removed.
    """
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    no_url = url_pattern.sub(r'', txt)
    return no_url
def show_graph(number_of_ngrams):
    bigram_df = pd.DataFrame(number_of_ngrams,columns=['ngram', 'count'])
    d = bigram_df.set_index('ngram').T.to_dict('records')
    # Create network plot 
    G = nx.Graph()
    # Create connections between nodes
    for k, v in d[0].items():
        G.add_edge(k[0], k[1], weight=(v * 10))
    fig, ax = plt.subplots(figsize=(10, 8))
    pos = nx.spring_layout(G, k=2)
    # Plot networks
    nx.draw_networkx(G, pos,font_size=16,width=3,edge_color='grey',node_color='purple',with_labels = False,ax=ax)
    # Create offset labels
    for key, value in pos.items():
        x, y = value[0]+.135, value[1]+.045
        ax.text(x, y,s=key,bbox=dict(facecolor='red', alpha=0.25),horizontalalignment='center', fontsize=13) 
    plt.show()

def show_ngrams(terms_ngram):
    # terms_bigram = [list(bigrams(tweet)) for tweet in tweets_nsw_nc]
    # View bigrams for the first tweet
    # terms_bigram[0]
    # Flatten list of bigrams in clean tweets
    ngrams_list = list(itertools.chain(*terms_ngram))

    # Create counter of words in clean bigrams
    ngrams_counts = collections.Counter(ngrams_list)

    tup = ngrams_counts.most_common(80)
    tup_graph = ngrams_counts.most_common(40) #use this for the networkx graph
    list1=[]
    list2=[]
    for item in tup:
        list1.append(' '.join(item[0]))    
        list2.append(item[1])
    lst_tuple = list(zip(list1,list2))
    print(lst_tuple)
    ngrams_df2 = pd.DataFrame(lst_tuple,columns=['ngrams', 'count'])
    df_to_dict  = ngrams_df2.set_index('ngrams')['count'].to_dict() #use this method
    wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(df_to_dict)
    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud)

    show_graph(tup_graph)

def word_cloud (df):
    # Remove URLs
    tweets_no_urls = []
    for i in range(len(df)):
        tweets_no_urls.append(remove_url(df.iloc[i]['tweet']))
    # Create a sublist of lower case words for each tweet
    words_in_tweet = [tweet.lower().split() for tweet in tweets_no_urls]

    stop_words = set(stopwords.words('english'))

    # Remove stop words from each tweet list of words
    tweets_nsw = [[word for word in tweet_words if not word in stop_words] for tweet_words in words_in_tweet]

    # Remove collection words
    collection_words = ['coronavirus', 'covid-19', 'tested', 'positive']

    tweets_nsw_nc = [[w for w in word if not w in collection_words] for word in tweets_nsw]
    tweets_nsw_nc = [x for x in tweets_nsw_nc if x != []]
    from nltk import bigrams, trigrams
    #displaying bigram word cloud
    terms_bigram = [list(bigrams(tweet)) for tweet in tweets_nsw_nc]
    show_ngrams(terms_bigram)
    # tweets_nsw_nc = [x for x in tweets_nsw_nc if x != []]
    # displaying trigram word cloud
    terms_trigram = [list(trigrams(tweet)) for tweet in tweets_nsw_nc if len(tweet) >2]
    show_ngrams(terms_trigram)  

### Rumour Word Cloud

In [ ]:
word_cloud(df_rumour)

 ### Non-Rumour Word Cloud

In [ ]:
word_cloud(df_non_rumour)

# **Question 4 - User Analysis - TABLE 4**

## Numeric Columns

#### Followers count

In [ ]:
total_nonRum_followers_count = df_non_rumour['followers_count'].sum()
print ("The sum of followers count is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['followers_count'].mean()
print ("The mean of non-rumours followers count is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['followers_count'].median()
print ("The median of non-rumours followers count is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['followers_count'].sum()
print ("The sum of followers count is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['followers_count'].mean()
print ("The mean of rumours followers count is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['followers_count'].median()
print ("The median of rumours followers count is: ",median_Rum_followers_count, "\n")

#### Friends Count

In [ ]:
total_nonRum_followers_count = df_non_rumour['friends_count'].sum()
print ("The sum of followers count is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['friends_count'].mean()
print ("The mean of non-rumours followers count is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['friends_count'].median()
print ("The median of non-rumours followers count is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['friends_count'].sum()
print ("The sum of followers count is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['friends_count'].mean()
print ("The mean of rumours followers count is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['friends_count'].median()
print ("The median of rumours followers count is: ",median_Rum_followers_count, "\n")

#### Statuses_Count

In [ ]:
total_nonRum_followers_count = df_non_rumour['statuses_count'].sum()
print ("The sum of statuses_count is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['statuses_count'].mean()
print ("The mean of non-rumours statuses_count is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['statuses_count'].median()
print ("The median of non-rumours statuses_count is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['statuses_count'].sum()
print ("The sum of statuses_count is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['statuses_count'].mean()
print ("The mean of rumours statuses_count is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['statuses_count'].median()
print ("The median of rumours statuses_count is: ",median_Rum_followers_count, "\n")

#### Listed_Count

In [ ]:
total_nonRum_followers_count = df_non_rumour['listed_count'].sum()
print ("The sum of listed_count is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['listed_count'].mean()
print ("The mean of non-rumours listed_count is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['listed_count'].median()
print ("The median of non-rumours listed_count is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['listed_count'].sum()
print ("The sum of listed_count is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['listed_count'].mean()
print ("The mean of rumours listed_count is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['listed_count'].median()
print ("The median of rumours listed_count is: ",median_Rum_followers_count, "\n")

#### Favourite_Count

In [ ]:
total_nonRum_followers_count = df_non_rumour['favorite_count'].sum()
print ("The sum of favorite_count is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['favorite_count'].mean()
print ("The mean of non-rumours favorite_count is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['favorite_count'].median()
print ("The median of non-rumours favorite_count is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['favorite_count'].sum()
print ("The sum of favorite_count is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['favorite_count'].mean()
print ("The mean of rumours favorite_count is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['favorite_count'].median()
print ("The median of rumours favorite_count is: ",median_Rum_followers_count, "\n")

#### Favourites_Count

In [ ]:
total_nonRum_followers_count = df_non_rumour['favourites_count'].sum()
print ("The sum of favourites_count is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['favourites_count'].mean()
print ("The mean of non-rumoursfavourites_count is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['favourites_count'].median()
print ("The median of non-rumours favourites_count is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['favourites_count'].sum()
print ("The sum of favourites_count is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['favourites_count'].mean()
print ("The mean of rumours favourites_count is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['favourites_count'].median()
print ("The median of rumours favourites_count is: ",median_Rum_followers_count, "\n")

#### Retweet_Count 

In [ ]:
total_nonRum_followers_count = df_non_rumour['retweet_count'].sum()
print ("The sum of retweet_count is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['retweet_count'].mean()
print ("The mean of non-rumours retweet_count is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['retweet_count'].median()
print ("The median of non-rumours retweet_count is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['retweet_count'].sum()
print ("The sum of retweet_count is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['retweet_count'].mean()
print ("The mean of rumours retweet_count is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['retweet_count'].median()
print ("The median of rumours retweet_count is: ",median_Rum_followers_count, "\n")

#### User Account Age

In [ ]:
total_nonRum_followers_count = df_non_rumour['account age (seconds)'].sum()
print ("The sum of account age (seconds) is: ",total_nonRum_followers_count)
mean_nonRum_followers_count = df_non_rumour['account age (seconds)'].mean()
print ("The mean of non-rumours account age (seconds) is: ", mean_nonRum_followers_count)
median_nonRum_followers_count = df_non_rumour['account age (seconds)'].median()
print ("The median of non-rumours account age (seconds) is: ",median_nonRum_followers_count, "\n")

total_Rum_followers_count = df_rumour['account age (seconds)'].sum()
print ("The sum of account age (seconds) is: ",total_Rum_followers_count)
mean_Rum_followers_count = df_rumour['account age (seconds)'].mean()
print ("The mean of rumours account age (seconds) is: ", mean_Rum_followers_count)
median_Rum_followers_count = df_rumour['account age (seconds)'].median()
print ("The median of rumours account age (seconds) is: ",median_Rum_followers_count, "\n")

In [ ]:
intervals = (
    ('years',)
    ('months',)
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])

In [ ]:
display_time(361935622)

In [ ]:
display_time(362529869)

In [ ]:
def display_age(secondss):
  years = secondss/ 60/60/24/265.25
  return years
print("Rumour age",display_age(362529869), "years")
print("NON-Rumour age",display_age(361935622), "years")

## Binary Columns

#### Binary Columns - geo-enabled

In [ ]:
print(df_rumour["geo_enabled"].value_counts(normalize=True).mul(100).astype(str)+'%')
print(df_non_rumour["geo_enabled"].value_counts(normalize=True).mul(100).astype(str)+'%')

#### Binary Columns - protected

In [ ]:
print(df_rumour["protected"].value_counts(normalize=True).mul(100).astype(str)+'%')
print(df_non_rumour["protected"].value_counts(normalize=True).mul(100).astype(str)+'%')

#### Binary Columns -  default_profile

In [ ]:
print(df_rumour["default_profile"].value_counts(normalize=True).mul(100).astype(str)+'%')
print(df_non_rumour["default_profile"].value_counts(normalize=True).mul(100).astype(str)+'%')

#### Binary Columns - verified

In [ ]:
print(df_rumour["verified"].value_counts(normalize=True).mul(100).astype(str)+'%')
print(df_non_rumour["verified"].value_counts(normalize=True).mul(100).astype(str)+'%')

## END OF ANALYSIS